In [ ]:
#import
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
from IPython.display import HTML
import re

In [ ]:
#scrap data
r=requests.get('https://www.wongnai.com/_api/v1.6/cooking/recipes/tags/fruit-recipes?_v=5.132&locale=th&page.number=1&page.size=1500')
obj=json.loads(r.text)
alldata = obj['data']['page']['entities']

In [ ]:
#data for dataframe
menu=[]
ing=[]
view=[]
link=[]
img=[]
for i in alldata:
    menu.append(i['title'])
    ing.append(i['ingredientKeywords'])
    view.append(i['view'])
    imgkey=i['photo']['smallUrl']
    img.append(f'<img src="{imgkey}" />')
for i in alldata:
    name=i['title']
    k=i['url']
    url=f'https://www.wongnai.com/{k}'
    linkkey=f'<a href="{url}">{name}</a>'
    link.append(linkkey)

In [ ]:
#first table
table = pd.DataFrame({'menu': menu, 'ingredient': ing,
                      'view': view, 'link':link, 'photo':img})

In [ ]:
fruit_c=[]

In [ ]:
#fruit data med
r=requests.get('https://medthai.com/%E0%B8%A3%E0%B8%B2%E0%B8%A2%E0%B8%8A%E0%B8%B7%E0%B9%88%E0%B8%AD%E0%B8%9C%E0%B8%A5%E0%B9%84%E0%B8%A1%E0%B9%89/')
s=BeautifulSoup(r.text, 'lxml')
#all_letter=['ก','ข','ค','ฆ','ง','จ','ฉ','ช','ซ','ฌ','ญ','ฎ','ฏ','ฐ','ฑ','ฒ','ณ','ด','ต','ถ','ท','ธ','น','บ','ป','ผ','ฝ','พ','ฟ','ภ','ม','ย','ร','ล','ว','ศ','ษ','ส','ห','ฬ','อ','ฮ']
letter=['ก','ข','ค','ง','จ','ฉ','ช','ซ','ด','ต','ถ','ท','น','บ','ป','ผ','ฝ','พ','ฟ','ม','ย','ร','ล','ว','ส','ห','อ']
mfruit_list=[]
for i in letter:
  key_letter=f'หมวด-{i}'
  d=s.find('div', {'id':key_letter})
  a_tags=d.find_all('a')
  k=a_tags
  for j in k:
    mfruit_list.append(j['href'])

mfruit =  [i.replace('/','') for i in mfruit_list]

#scrap ชื่อผลไม้ wiki
r=requests.get('https://th.wikipedia.org/wiki/%E0%B8%A3%E0%B8%B2%E0%B8%A2%E0%B8%81%E0%B8%B2%E0%B8%A3%E0%B8%9C%E0%B8%A5%E0%B9%84%E0%B8%A1%E0%B9%89')
s=BeautifulSoup(r.text, 'lxml')
d=s.find('div', {'id':'mw-content-text'})
a_tags=d.find_all('a')
tag=a_tags
wfruit_list=[]
wfruit=[]
for i in range(20,199):
  wfruit_list.append(tag[i]['title'])
delete=['แก้ไขส่วน: ข','แก้ไขส่วน: ค','แก้ไขส่วน: ง','แก้ไขส่วน: ช','แก้ไขส่วน: ต','แก้ไขส่วน: ท','แก้ไขส่วน: น','แก้ไขส่วน: บ','แก้ไขส่วน: ฝ','แก้ไขส่วน: พ','แก้ไขส่วน: ม','แก้ไขส่วน: ร','แก้ไขส่วน: ล','แก้ไขส่วน: ส','แก้ไขส่วน: ห','แก้ไขส่วน: อ','หมวดหมู่:',' (พืช)',' (ผลไม้)']
for i in wfruit_list:
    for j in delete:
        wfruit_list=[x.replace(j,'') for x in wfruit_list]
for i in wfruit_list:
    if len(i)!=0:
        wfruit.append(i)

#combine fruit
fruit_c = list(set().union(wfruit,mfruit))
fruit_c

In [ ]:
#edit fruit
#remove from fruit
fruit_c=[k.replace('แอปเปิ้ลไซเดอร์','แอปเปิลไซเดอร์') for k in fruit_c]
fruit_c=[k.replace('เบอร์รี่','เบอร์รี') for k in fruit_c]
for i in fruit_c:
    for j in ['สตรอเบอร์รี','สตรอเบอร์รี่','กีวี่','แอปเปิ้ล','เบอรี่','เชอร์รี่','ลูกพลับ']:
        fruit_c=[k.replace(j,'') for k in fruit_c]
fruit_c

In [ ]:
#add fruit
fruit=[]
for i in fruit_c:
    if len(i)>1:
        fruit.append(i)
for j in ['พีช','เมล่อน','สตรอว์เบอร์รี','มะยงชิด','เบอร์รี']:
        fruit.append(j)
fruit

In [ ]:
#rectify word in table
table_del=['น้ำตาล','น้ำตาลทราย','น้ำตาลปี๊บ','จากสวนหลังบ้าน','จากกาแฟ','แก้วกาแฟ','สไปรูลิน่า','flaxseeds จาก superfoods',' flax seeds จาก superfoods','spirulina powder จาก superfoods','kale powder จาก superfoods','chia seeds จาก superfoods','flaxseeds จาก superfoods','acai powder จาก superfoods','maca powerder จาก superfoods']
table_straw=['สตรอว์เบอร์รี่','สตรอว์เบอรี่','สตรอว์เบอรี','สตรอเบอร์รี่','สตรอเบอร์รี','สตรอเบอรี่','สตรอเบอรี','สตอเบอรี่','สตอเบอรี','สตอเบอร์รี่','สตอเบอร์รี']
for i in table_del:
    table['ingredient']=table['ingredient'].str.replace(i,'')
for i in table_straw:
    table['ingredient']=table['ingredient'].str.replace(i,'สตรอว์เบอร์รี')
table['ingredient']=table['ingredient'].str.replace('แอปเปิ้ล','แอปเปิล').str.replace('เบอรี่','เบอร์รี').str.replace('เบอร์รี่','เบอร์รี').str.replace('เปลือกแก้งมังกร','เปลือกแก้วมังกร').str.replace('กล้วยน้ำหว้า','กล้วยน้ำว้า').str.replace('สัปปะรด','สับปะรด').str.replace('อินทผาลัม','อินทผลัม').str.replace('กรีวี่','กีวี').str.replace('กีวี่','กีวี').str.replace('แตงไท','แตงไทย')
table_ing=table['ingredient']
table_ing

In [ ]:
#ing_find=table['ingredient']
ing_find=table_ing
fruit_have=[]
for i in ing_find:
    list_fruit=''
    for j in fruit:
        x=re.findall(j,i)
        if x != []:
            if list_fruit=='':
                list_fruit=j
            else:
                list_fruit=list_fruit+','+j
    if list_fruit != '':
        fruit_have.append(list_fruit)
    else:
        fruit_have.append('-')

In [ ]:
#create dataframe
table = pd.DataFrame({'menu': menu, 'ingredient': table_ing,'fruit': fruit_have,
                      'view': view, 'link':link, 'photo':img})
pd.set_option('display.max_colwidth', -1)
table['ingredient'] =  table['ingredient'].apply(lambda x: re.sub(r"[^\u0E00-\u0E7Fa-zA-Z' ,]|^'|'$|''",'', str(x))) #remove emoji
HTML(table.to_html(escape=False))

In [ ]:
fcount=(table.set_index(['menu','ingredient','view','link','photo']).apply(lambda x: x.str.split(',').explode()).reset_index())
fcount

In [ ]:
fcount['fruit'].value_counts().head(60)

In [ ]:
fcount['fruit'].value_counts().tail(60)

In [ ]:
fcount

In [ ]:
#remove from table
'น้ำตาล','น้ำตาลทราย','น้ำตาลปี๊บ','จากสวนหลังบ้าน','จากกาแฟ','แก้วกาแฟ','สไปรูลิน่า','flaxseeds จาก superfoods',' flax seeds จาก superfoods','spirulina powder จาก superfoods','kale powder จาก superfoods','chia seeds จาก superfoods','flaxseeds จาก superfoods','acai powder จาก superfoods','maca powerder จาก superfoods'
'สตรอว์เบอร์รี่','สตรอว์เบอรี่','สตรอว์เบอรี','สตรอเบอร์รี่','สตรอเบอร์รี','สตรอเบอรี่','สตรอเบอรี','สตอเบอรี่','สตอเบอรี','สตอเบอร์รี่','สตอเบอร์รี'

'เเอปเปิ้ล','แอปเปิล'
'เบอรี่','เบอร์รี'
'เบอร์รี่','เบอร์รี'
'เปลือกแก้งมังกร','เปลือกแก้วมังกร'
'กล้วยน้ำหว้า','กล้วยน้ำว้า'
'สัปปะรด','สับปะรด'
'อินทผาลัม','อินทผลัม'
'กรีวี่','กีวี'
'กีวี่','กีวี'
'แตงไท','แตงไทย'

#remove from fruit
'สตรอเบอร์รี','สตรอเบอร์รี่','กีวี่','แอปเปิ้ล'
'แอปเปิ้ลไซเดอร์','แอปเปิลไซเดอร์'
'เบอรี่','เบอร์รี่','เบอร์รี'

#append to fruit
'พีช','ผลไม้','เมล่อน','สตรอว์เบอร์รี'

In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
f_input=input()
f_test=table['fruit']
s_menu=[]
s_ing=[]
s_fruit_have=[]
s_view=[]
s_link=[]
s_img=[]

header=[s_menu,s_ing,s_fruit_have,s_view,s_link,s_img]
header_a=['menu','ingredient','fruit','view','link','photo']
for i in range(len(table)):
    if fuzz.token_sort_ratio(f_test[i],f_input)>=70:
        for j in range(6):
            add=table[header_a[j]]
            header[j].append(add[i])
s_table = pd.DataFrame({'menu': s_menu, 'ingredient': s_ing,'fruit': s_fruit_have,
                      'view': s_view, 'link':s_link, 'photo':s_img})
table_rec=s_table.sort_values(['view'],ascending=False).reset_index()
HTML(table_rec[['link','photo']].to_html(escape=False))

In [ ]:
def simple():
    f_input=input()
    f_test=table['fruit']
    s_menu=[]
    s_ing=[]
    s_fruit_have=[]
    s_view=[]
    s_link=[]
    s_img=[]

    header=[s_menu,s_ing,s_fruit_have,s_view,s_link,s_img]
    header_a=['menu','ingredient','fruit','view','link','photo']
    for i in range(len(table)):
        if fuzz.token_sort_ratio(f_test[i],f_input)>=70:
            for j in range(6):
                add=table[header_a[j]]
                header[j].append(add[i])
    s_table = pd.DataFrame({'menu': s_menu, 'ingredient': s_ing,'fruit': s_fruit_have,
                      'view': s_view, 'link':s_link, 'photo':s_img})
    table_rec=s_table.sort_values(['view'],ascending=False).reset_index()
    return HTML(table_rec[['link','photo']].to_html(escape=False))
simple()

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
def simple(kk):
    f_input=kk
    f_test=table['fruit']
    s_menu=[]
    s_ing=[]
    s_fruit_have=[]
    s_view=[]
    s_link=[]
    s_img=[]

    header=[s_menu,s_ing,s_fruit_have,s_view,s_link,s_img]
    header_a=['menu','ingredient','fruit','view','link','photo']
    for i in range(len(table)):
        if fuzz.token_sort_ratio(f_test[i],f_input)>=70:
            for j in range(6):
                add=table[header_a[j]]
                header[j].append(add[i])
    s_table = pd.DataFrame({'menu': s_menu, 'ingredient': s_ing,'fruit': s_fruit_have,
                      'view': s_view, 'link':s_link, 'photo':s_img})
    table_rec=s_table.sort_values(['view'],ascending=False).reset_index()
    return HTML(table_rec[['link','photo']].to_html(escape=False))
interact(simple, kk=['ลิ้นจี่','ลำไย','แอปเปิล']);

In [ ]:
import ipywidgets as widgets
from IPython.core.display import display, HTML, Markdown, clear_output
data = ['ลิ้นจี่', 'ลำไย', 'แอปเปิล', 'ทุเรียน','กล้วย','มะม่วง','มะพร้าว','เสาวรส','ทุเรียน','แตงโม','ข้าวโพด','มะนาว','ส้ม','สับปะรด','แอปเปิล','ฟักทอง','กระท้อน','มังคุด','สตรอว์เบอร์รี','-']
checkboxes = [widgets.Checkbox(value=False, description=label) for label in data]
vbox = widgets.VBox(children=checkboxes)
button=widgets.Button(description='find')
out = widgets.Output(layout={'border': '1px solid black'})

def find(b):
    selected_data = []
    for i in range(len(checkboxes)):
        if checkboxes[i].value == True:
            selected_data.append(checkboxes[i].description)
    f_input=selected_data
    f_test=table['fruit']
    s_menu=[]
    s_ing=[]
    s_fruit_have=[]
    s_view=[]
    s_link=[]
    s_img=[]
    header=[s_menu,s_ing,s_fruit_have,s_view,s_link,s_img]
    header_a=['menu','ingredient','fruit','view','link','photo']
    for i in range(len(table)):
        if fuzz.partial_ratio(f_test[i],f_input)>=70:
            for j in range(6):
                add=table[header_a[j]]
                header[j].append(add[i])
    s_table = pd.DataFrame({'menu': s_menu, 'ingredient': s_ing,'fruit': s_fruit_have,
                      'view': s_view, 'link':s_link, 'photo':s_img})
    table_rec=s_table.sort_values(['view'],ascending=False).reset_index()
    ttt = HTML(table_rec[['link','photo']].to_html(escape=False))
    if len(s_table)>0:
        with out:
            out.clear_output()
            #clear_output(True)
            display(ttt)
    else:
        out.clear_output()
        out.append_stdout('No recipe for you')

    
button.on_click(find)

display(vbox,button,out)

In [ ]:
import ipywidgets as widgets
from IPython.core.display import display, HTML, Markdown, clear_output
data = ["ลิ้นจี่", "ลำไย", "แอปเปิล", "ทุเรียน",'-']
checkboxes = [widgets.Checkbox(value=False, description=label) for label in data]
vbox = widgets.VBox(children=checkboxes)
button=widgets.Button(description='find')
out = widgets.Output(layout={'border': '1px solid black'})

        
def simple(fhave):
    f_input=fhave
    f_test=table['fruit']
    s_menu=[]
    s_ing=[]
    s_fruit_have=[]
    s_view=[]
    s_link=[]
    s_img=[]

    header=[s_menu,s_ing,s_fruit_have,s_view,s_link,s_img]
    header_a=['menu','ingredient','fruit','view','link','photo']
    for i in range(len(table)):
        if fuzz.partial_ratio(f_test[i],f_input)>=70:
            for j in range(6):
                add=table[header_a[j]]
                header[j].append(add[i])
    s_table = pd.DataFrame({'menu': s_menu, 'ingredient': s_ing,'fruit': s_fruit_have,
                      'view': s_view, 'link':s_link, 'photo':s_img})
    table_rec=s_table.sort_values(['view'],ascending=False).reset_index()
    ttt = HTML(table_rec[['link','photo']].to_html(escape=False))
    with out:
        clear_output(True)
        display(ttt)    

def find(b):
    selected_data = []
    for i in range(len(checkboxes)):
        if checkboxes[i].value == True:
            selected_data.append(checkboxes[i].description)
    simple(selected_data)

button.on_click(find)
display(vbox,button,out)

In [ ]:
import pandas as pd                                                                                                                                

state_df = pd.DataFrame({'state':['Smallมากๆ', 'Medium', 'High', 'Small', 'High','Max']})                                                                    

state_df

In [ ]:
replace_values = {'Small' : 'เล็ก', 'Medium' : 2, 'High' :3,'Max' : 3 }                                                                                          

state_df = state_df.replace({"state": replace_values})                                                                                             

state_df

In [ ]:
state_df = state_df.replace('Small', '1').replace('Medium', '5').replace('High', '15')
state_df

In [ ]:
state_df.apply(lambda x: x.str.replace('Small', '1').str.replace('Medium', '5'))

In [ ]:
words = ['how', 'much', 'is[br]', 'the', 'fish[br]', 'no', 'really']

In [ ]:
words = [word.replace('[br]','<br />') for word in words]

In [ ]:
words